## 通过 Qwen Agent 操作 FastAPI

- GitHub: [tadata-org/fastapi_mcp](https://github.com/tadata-org/fastapi_mcp)
- Docs: [getting-started](https://fastapi-mcp.tadata.com/getting-started/welcome)
- Examples: [examples](https://github.com/tadata-org/fastapi_mcp/tree/main/examples)

In [1]:
# !uv pip install fastapi-mcp

In [2]:
import uvicorn

from fastapi import FastAPI
from fastapi_mcp import FastApiMCP
from qwen_agent.agents import Assistant

FastAPI 服务端代码见 [fastapi_mcp_server.py](./fastapi_mcp_server.py)

启动 FastAPI 服务：

```bash
python fastapi_mcp_server.py
```

运行客户端代码：

In [4]:
def init_agent_service():
    llm_cfg = {
        'model': 'Qwen3-0.6B-FP8',
        'model_server': 'http://localhost:8000/v1',
        'api_key': 'token-kcgyrk',
        'generate_cfg': {
            'top_p': 0.95,
            'temperature': 0.6,
        }
    }
    system = ('你扮演一个 API 调用助手，你具有查询用户信息的能力')
    tools = [{
      "mcpServers": {
        "fastapi-mcp": {
          "url": "http://localhost:8351/mcp"
        }
      }
    }]

    bot = Assistant(
        llm=llm_cfg,
        name='API 调用助手',
        description='用户信息查询',
        system_message=system,
        function_list=tools,
    )

    return bot

In [5]:
# 初始化 Agent
bot = init_agent_service()

2025-05-20 11:35:45,512 - mcp_manager.py - 110 - INFO - Initializing MCP tools from mcp servers: ['fastapi-mcp']


In [6]:
query = '用户Nagisa的ID是9523，请获取该用户的用户信息'
messages = [{'role': 'user', 'content': query}]

# 输出
response = bot.run_nonstream(messages)
print('bot response:', response)

bot response: [{'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户需要获取用户Nagisa的ID为9523的信息。首先，我应该检查可用的工具，看看有没有相关的函数。提供的工具中有一个fastapi-mcp-get_user_info，它的参数需要user_id，类型是整数，且必须提供。用户已经明确给出了ID，所以直接调用这个函数，把user_id设为9523。不需要其他步骤，应该就能得到用户的信息了。\n', 'name': 'API 调用助手'}, {'role': 'assistant', 'content': '', 'name': 'API 调用助手', 'function_call': {'name': 'fastapi-mcp-get_user_info', 'arguments': '{"user_id": 9523}'}}, {'role': 'function', 'content': '{\n  "user_id": 9523,\n  "nationality": "China",\n  "birthday": "2.12"\n}', 'name': 'fastapi-mcp-get_user_info'}, {'role': 'assistant', 'content': '', 'reasoning_content': '\n好的，用户之前让我获取Nagisa的用户信息，我调用了fastapi-mcp-get_user_info函数，参数是用户ID9523。现在API返回的信息显示用户ID是9523，国籍是中国，生日是2月12日。我需要确认用户是否需要这些信息，或者是否有其他需求。用户的问题明确提到获取Nagisa的信息，所以应该直接提供这些数据。检查是否有其他工具需要调用，但目前只有这个函数可用，所以不需要额外操作。最后，将结果整理成自然的中文回复。\n', 'name': 'API 调用助手'}, {'role': 'assistant', 'content': '\n\n用户Nagisa的信息如下：\n\n- 用户ID：9523\n- 国籍：中国\n- 生日：2月12日\n\n如果需要进一步帮助，请随时告诉我！', 'name': 'API 

In [7]:
print('result:', response[-1]['content'].strip())

result: 用户Nagisa的信息如下：

- 用户ID：9523
- 国籍：中国
- 生日：2月12日

如果需要进一步帮助，请随时告诉我！
